In [1]:
# The code was removed by Watson Studio for sharing.

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Ready


In [2]:
body = client_6b26c2b97158494a896c5437154f01aa.get_object(Bucket='capstoneproject-donotdelete-pr-idqhne3njlglvj',Key='Alberghi.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

In [3]:
df_hotel_base = pd.read_csv(body, sep=";", encoding='latin1')
df_hotel_base.head()

,INSEGNA,CLASSIFICAZIONE,INDIRIZZO,MUNICIPIO,TIPOLOGIA,SINGOLE,DOPPIE,TRIPLE,QUADRUPLE,QUINTUPLE,SESTUPLE,TOTALE_CAMERE,POSTI_LETTO,UNITA_ABITATIVE,POSTI_LETTO_UNITA_ABITATIVE
0,affittacamere navona central suites,Categoria 3,"VIA DELLA SCROFA, 22",Municipio I (ex Mun I),Guest House o Affittacamere,1,1,2,0,0,0,4,9,0,0
1,CAPPELLARI29,Categoria 2,"VIA DEI CAPPELLARI, 29",Municipio I (ex Mun I),Casa Vacanze NON impr (Appartamento),0,0,0,0,0,0,0,0,1,1
2,CASA VACANZE SEMINARIO -,Categoria 2,"VIA DEL SEMINARIO, 85",Municipio I (ex Mun I),Casa Vacanze NON impr (Appartamento),0,3,0,0,0,0,3,6,0,0
3,A BEAUTIFUL PLACE IN ROME,Categoria 3,"VIA VICENZA, 20",Municipio I (ex Mun I),Guest House o Affittacamere,0,3,0,0,0,0,3,6,0,0
4,A CASA A TESTACCIO,Unica,"VIA GALVANI, 4",Municipio I (ex Mun I),Bed & Breakfast non imprenditoriale,0,0,0,0,0,0,2,4,0,0


In [4]:
df_hotel = df_hotel_base
df_hotel = df_hotel.drop(columns=['POSTI_LETTO_UNITA_ABITATIVE'])
df_hotel = df_hotel.drop(columns=['UNITA_ABITATIVE'])
df_hotel = df_hotel.drop(columns=['CLASSIFICAZIONE'])
df_hotel = df_hotel[df_hotel.POSTI_LETTO!=0]
df_hotel_I = df_hotel
df_hotel_I = df_hotel[df_hotel.MUNICIPIO=="Municipio I (ex Mun I)"]
df_hotel_I = df_hotel[df_hotel.INDIRIZZO!=""]
df_hotel_I["LONGITUDINE"] = 0
df_hotel_I["LATITUDINE"] = 0
df_hotel_I.index = pd.RangeIndex(len(df_hotel_I.index))
df_hotel_I.head()

,INSEGNA,INDIRIZZO,MUNICIPIO,TIPOLOGIA,SINGOLE,DOPPIE,TRIPLE,QUADRUPLE,QUINTUPLE,SESTUPLE,TOTALE_CAMERE,POSTI_LETTO,LONGITUDINE,LATITUDINE
0,affittacamere navona central suites,"VIA DELLA SCROFA, 22",Municipio I (ex Mun I),Guest House o Affittacamere,1,1,2,0,0,0,4,9,0,0
1,CASA VACANZE SEMINARIO -,"VIA DEL SEMINARIO, 85",Municipio I (ex Mun I),Casa Vacanze NON impr (Appartamento),0,3,0,0,0,0,3,6,0,0
2,A BEAUTIFUL PLACE IN ROME,"VIA VICENZA, 20",Municipio I (ex Mun I),Guest House o Affittacamere,0,3,0,0,0,0,3,6,0,0
3,A CASA A TESTACCIO,"VIA GALVANI, 4",Municipio I (ex Mun I),Bed & Breakfast non imprenditoriale,0,0,0,0,0,0,2,4,0,0
4,A CASA CARRA,"VIA DELLO STATUTO, 44",Municipio I (ex Mun I),Guest House o Affittacamere,2,1,0,0,0,0,3,4,0,0


In [5]:
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode('ROMA')
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

41.894802 12.4853384


In [10]:
map_rome = folium.Map(location=[latitude, longitude], zoom_start=12)
map_rome

In [25]:
# The code was removed by Watson Studio for sharing.

In [46]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    RADIUS, 
    LIMIT)
            
results = requests.get(url).json()["response"]['groups'][0]['items']

venues_list=[]

venues_list.append([(
    v['venue']['name'], 
    v['venue']['location']['lat'], 
    v['venue']['location']['lng'],  
    v['venue']['categories'][0]['name']) for v in results])

nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues.columns = ['Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

In [47]:
nearby_venues.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Foro di Traiano,41.894729,12.484871,Historic Site
1,Colonna Traiana,41.895816,12.484276,Monument / Landmark
2,Foro di Cesare,41.894128,12.485232,Historic Site
3,Terrazza delle Quadrighe,41.894346,12.483336,Scenic Lookout
4,Piazza Venezia,41.895469,12.482664,Plaza


In [48]:
map_rome = folium.Map(location=[latitude, longitude], zoom_start=16)

for lat, lng, venue, venue_c in zip(nearby_venues['Venue Latitude'], nearby_venues['Venue Longitude'], nearby_venues['Venue'], nearby_venues['Venue Category']):
    label = '{}, {}'.format(venue, venue_c)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rome)  
    
map_rome

In [69]:
from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler

nearby_venues_cluster = nearby_venues

sklearn.utils.check_random_state(1000)
Clus_dataSet = nearby_venues[['Venue Latitude','Venue Longitude']]
Clus_dataSet = np.nan_to_num(Clus_dataSet)
Clus_dataSet = StandardScaler().fit_transform(Clus_dataSet)

# Compute DBSCAN
db = DBSCAN(eps=0.4, min_samples=5).fit(Clus_dataSet)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_
nearby_venues_cluster["LABEL"]=labels

realClusterNum=len(set(labels)) - (1 if -1 in labels else 0)
clusterNum = len(set(labels)) 


# A sample of clusters
nearby_venues_cluster.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,LABEL
0,Foro di Traiano,41.894729,12.484871,Historic Site,0
1,Colonna Traiana,41.895816,12.484276,Monument / Landmark,0
2,Foro di Cesare,41.894128,12.485232,Historic Site,0
3,Terrazza delle Quadrighe,41.894346,12.483336,Scenic Lookout,0
4,Piazza Venezia,41.895469,12.482664,Plaza,0


In [70]:
map_rome = folium.Map(location=[latitude, longitude], zoom_start=16)

x = np.arange(clusterNum)
ys = [i+x+(i*x)**2 for i in range(clusterNum)]
c = cm.rainbow(np.linspace(0, 1, len(ys)))
r = [colors.rgb2hex(i) for i in c]

markers_colors = []

for lat, lon, cluster in zip(nearby_venues_cluster['Venue Latitude'], nearby_venues_cluster['Venue Longitude'], nearby_venues_cluster['LABEL']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=r[cluster-1],
        fill=True,
        fill_color=r[cluster-1],
        fill_opacity=0.7).add_to(map_rome)
    
map_rome

In [76]:
nearby_venues_cluster.groupby('LABEL').size()

LABEL
-1    23
 0    19
 1    36
 2     5
 3     6
 4    11
dtype: int64

In [79]:
winning_place = nearby_venues_cluster
winning_place = winning_place[winning_place.LABEL==1]
winning_place.head()

,Venue,Venue Latitude,Venue Longitude,Venue Category,LABEL
7,Via dei Fori Imperiali,41.894036,12.487228,Road,1
15,Foro Romano,41.892117,12.486998,Historic Site,1
16,Templio de Antonino e Faustina,41.892037,12.486597,Temple,1
17,La Taverna dei Fori Imperiali,41.893989,12.488528,Italian Restaurant,1
20,Tempio del Divo Romolo,41.891863,12.487126,Temple,1


In [86]:
best_place_latitude = winning_place["Venue Latitude"].mean()
best_place_longitude = winning_place["Venue Longitude"].mean()

print("BEST PLACE IS {} {}".format(best_place_latitude,best_place_longitude))

BEST PLACE IS 41.894383146443865 12.490244910515004


In [ ]:
map_best_place = folium.Map(location=[best_place_latitude, best_place_longitude], zoom_start=22)

label = '{}, {}'.format(venue, venue_c)
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
[lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_rome)  
    
map_rome